In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

box_DIR = "../data/box/"
img_DIR_P = "../data/images/players/"
fig_DIR = "../figs/players/"

In [ ]:
season = 2024
df1 = pd.read_parquet(box_DIR + f"NBA_Box_P_Base_{season}.parquet")
df1["GAME_DATE"] = pd.to_datetime(df1["GAME_DATE"])

## Making 0 3s

In [ ]:
df1g = df1.groupby(["PLAYER_ID","PLAYER_NAME"])
keys = list(df1g.groups)
dfa = []
for key in keys:
    df2 = df1g.get_group(key)
    df2["ct"] = (df2["FG3A"] >0) & (df2["FG3M"] == 0)
    df2 = df2.sort_values("GAME_DATE")
    m = df2.ct
    df2['Games'] = (m.groupby([m, (~m).cumsum().where(m)]).cumcount().add(1).mul(m))
    df2['Games'] = df2['Games'].fillna(0)
    df2['Games'] = df2['Games'].astype(int)
    # df2 = df2.sort_values("Games").tail(1)
    dfa.append(df2)
df3 = pd.concat(dfa)

In [ ]:
# df3[["PLAYER_NAME","FG3M","FG3A","GAME_DATE"]].query("PLAYER_NAME == 'Bam Adebayo'").query("GAME_DATE >= '2024-12-20'")

In [ ]:
df3.columns

In [ ]:
df4 = df3[["PLAYER_ID","PLAYER_NAME","TEAM_ABBREVIATION","Games"]]
df4 = df4.sort_values("Games",ascending=False).reset_index(drop=True)
df4 = df4[~df4["PLAYER_ID"].duplicated(keep="first")]

In [ ]:
data = df4.reset_index(drop=True).reset_index().head(15)
data["PLAYER_ID"] = data["PLAYER_ID"].astype(str)
data["index"] +=1
t = (
        GT(data)
        .tab_header(
            title = md("**Consecutive Games with Missing All 3 Point Shots**"),
            subtitle = md(f"{get_ss(season)} NBA Season, Min 1 3PA in a game")  
        )
        .tab_source_note(source_note="bsky:@sradjoker.cc | X:@sradjoker | source: nba.com/stats")
        .cols_label(
            index = "#",
            PLAYER_ID = "",
            PLAYER_NAME = "Name",
            TEAM_ABBREVIATION="Team",
        )
        .fmt_image(
            columns="PLAYER_ID",
            path = img_DIR_P,
            file_pattern="{}.png"
        )
        .cols_align(align="center")
        .cols_align(align="left", columns="PLAYER_NAME")
        .tab_options(
            heading_title_font_size="150%",
            heading_subtitle_font_size="110%",
            # heading_title_font_weight='bold',
            table_background_color="floralwhite",
            column_labels_font_size="105%",
            column_labels_font_weight='bold',
            row_group_font_weight='bold',
            row_group_background_color="#E5E1D8",
            table_font_size=10,
            table_font_names="Consolas", 
            data_row_padding = "0.5px",
            # table_margin_left = 7,
            # table_margin_right = 0,
        )
    )
t.save(fig_DIR + f"ThreePoint_Missing_Streaks_{season}.png",scale=3,web_driver="firefox")
t

## Making 3 3s

In [ ]:
df1g = df1.groupby(["PLAYER_ID","PLAYER_NAME"])
keys = list(df1g.groups)
dfa = []
for key in keys:
    df2 = df1g.get_group(key)
    df2["ct"] = (df2["FG3A"] >0) & (df2["FG3M"] > 2)
    df2 = df2.sort_values("GAME_DATE")
    m = df2.ct
    df2['Games'] = (m.groupby([m, (~m).cumsum().where(m)]).cumcount().add(1).mul(m))
    df2['Games'] = df2['Games'].fillna(0)
    df2['Games'] = df2['Games'].astype(int)
    # df2 = df2.sort_values("Games").tail(1)
    dfa.append(df2)
df3 = pd.concat(dfa)

In [ ]:
df4 = df3[["PLAYER_ID","PLAYER_NAME","TEAM_ABBREVIATION","Games"]]
df4 = df4.sort_values("Games",ascending=False).reset_index(drop=True)
df4 = df4[~df4["PLAYER_ID"].duplicated(keep="first")]

In [ ]:
data = df4.reset_index(drop=True).reset_index().head(15)
data["PLAYER_ID"] = data["PLAYER_ID"].astype(str)
data["index"] +=1
t = (
        GT(data)
        .tab_header(
            title = md("**Consecutive Games with Making at least Three 3 Point Shots**"),
            subtitle = md(f"{get_ss(season)} NBA Season")  
        )
        .tab_source_note(source_note="bsky:@sradjoker.cc | X:@sradjoker | source: nba.com/stats")
        .cols_label(
            index = "#",
            PLAYER_ID = "",
            PLAYER_NAME = "Name",
            TEAM_ABBREVIATION="Team",
        )
        .fmt_image(
            columns="PLAYER_ID",
            path = img_DIR_P,
            file_pattern="{}.png"
        )
        .cols_align(align="center")
        .cols_align(align="left", columns="PLAYER_NAME")
        .tab_options(
            heading_title_font_size="150%",
            heading_subtitle_font_size="110%",
            # heading_title_font_weight='bold',
            table_background_color="floralwhite",
            column_labels_font_size="105%",
            column_labels_font_weight='bold',
            row_group_font_weight='bold',
            row_group_background_color="#E5E1D8",
            table_font_size=10,
            table_font_names="Consolas", 
            data_row_padding = "0.5px",
            # table_margin_left = 7,
            # table_margin_right = 0,
        )
    )
t.save(fig_DIR + f"ThreePoint_Making_Streaks_{season}.png",scale=3,web_driver="firefox")
t